In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd

# Preprocesado y modelado
# ==============================================================================
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsClassifier
import multiprocessing

# Descarga del target
# ==============================================================================
from google.colab import files

#Importando DataFram y observación de datos


In [ ]:
DataFrame = pd.read_csv('/content/Training.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DataFrame.shape

(16947, 52)

In [ ]:
DataFrame.describe()

,ID,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Opportunity_ID,ASP,ASP_(converted),Delivery_Year,TRF,Total_Amount,Total_Taxable_Amount
count,16947.000000,16947.000000,16947.000000,16947.000000,16947.000000,16947.0,16947.000000,13738.000000,13738.000000,16947.000000,16947.000000,1.688800e+04,1.694700e+04
mean,17666.650440,0.807990,0.583584,0.475482,0.318876,0.0,5335.940225,7.012128,0.432554,2017.137428,2.338172,1.993659e+06,3.224117e+06
std,6940.859372,0.393893,0.492979,0.499413,0.466055,0.0,3324.723809,18.858359,0.758659,0.829603,12.169339,1.848426e+07,2.030360e+07
min,4854.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,2016.000000,0.000000,0.000000e+00,0.000000e+00
25%,11953.000000,1.000000,0.000000,0.000000,0.000000,0.0,2448.500000,0.360000,0.373260,2016.000000,0.000000,2.677500e+04,8.546580e+04
50%,18114.000000,1.000000,1.000000,0.000000,0.000000,0.0,5306.000000,0.400000,0.412850,2017.000000,0.000000,1.072500e+05,3.760672e+05
75%,23845.500000,1.000000,1.000000,1.000000,1.000000,0.0,7698.000000,0.505000,0.476490,2018.000000,1.000000,4.603898e+05,1.037450e+06
max,28773.000000,1.000000,1.000000,1.000000,1.000000,0.0,12803.000000,80.000000,67.000000,2020.000000,500.000000,1.320000e+09,1.356339e+09


In [ ]:
DataFrame['Delivery_Year'].value_counts()

2017    6318
2018    5830
2016    4371
2019     414
2020      14
Name: Delivery_Year, dtype: int64

##Observo año 2016

In [ ]:
DataFrame[['Delivery_Year', 'Delivery_Quarter']][DataFrame['Delivery_Year'] == 2016]['Delivery_Quarter'].value_counts()

Q1    1264
Q2    1076
Q4    1032
Q3     999
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame[['Delivery_Year', 'Total_Amount']][DataFrame['Delivery_Year'] == 2016]['Total_Amount'].describe()

count    4.314000e+03
mean     1.996748e+06
std      1.107517e+07
min      0.000000e+00
25%      1.800000e+04
50%      1.160250e+05
75%      5.585332e+05
max      3.150001e+08
Name: Total_Amount, dtype: float64

##Observo año 2017

In [ ]:
DataFrame[['Delivery_Year', 'Delivery_Quarter']][DataFrame['Delivery_Year'] == 2017]['Delivery_Quarter'].value_counts()

Q2    1704
Q4    1662
Q3    1629
Q1    1323
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame[['Delivery_Year', 'Total_Amount']][DataFrame['Delivery_Year'] == 2017]['Total_Amount'].describe()

count    6.316000e+03
mean     1.552745e+06
std      1.442802e+07
min      0.000000e+00
25%      2.916639e+04
50%      1.044144e+05
75%      4.433962e+05
max      9.199924e+08
Name: Total_Amount, dtype: float64

##Observo año 2018

In [ ]:
DataFrame[['Delivery_Year', 'Delivery_Quarter']][DataFrame['Delivery_Year'] == 2018]['Delivery_Quarter'].value_counts()

Q2    1582
Q4    1435
Q1    1431
Q3    1382
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame[['Delivery_Year', 'Total_Amount']][DataFrame['Delivery_Year'] == 2018]['Total_Amount'].describe()

count    5.830000e+03
mean     1.532077e+06
std      1.152037e+07
min      0.000000e+00
25%      3.437719e+04
50%      1.015000e+05
75%      3.735200e+05
max      6.600053e+08
Name: Total_Amount, dtype: float64

##Observo año 2019

In [ ]:
DataFrame[['Delivery_Year', 'Delivery_Quarter']][DataFrame['Delivery_Year'] == 2019]['Delivery_Quarter'].value_counts()

Q1    290
Q2     77
Q3     32
Q4     15
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame[['Delivery_Year', 'Total_Amount']][DataFrame['Delivery_Year'] == 2019]['Total_Amount'].describe()

count    4.140000e+02
mean     8.926692e+06
std      4.391005e+07
min      7.500000e+02
25%      1.003105e+05
50%      8.766487e+05
75%      5.666865e+06
max      7.600000e+08
Name: Total_Amount, dtype: float64

##Observo año 2020 (Pero no lo voy a utilizar por poca cantidad de datos)

In [ ]:
DataFrame[['Delivery_Year', 'Delivery_Quarter']][DataFrame['Delivery_Year'] == 2020]['Delivery_Quarter'].value_counts()

Q1    9
Q2    4
Q3    1
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame[['Delivery_Year', 'Total_Amount']][DataFrame['Delivery_Year'] == 2020]['Total_Amount'].describe()

count    1.400000e+01
mean     1.871536e+08
std      3.776309e+08
min      2.842000e+05
25%      4.524957e+06
50%      2.505100e+07
75%      8.507118e+07
max      1.320000e+09
Name: Total_Amount, dtype: float64

#Forma 1 (One Hot Encoder a mano)


##Conversion a Dolar

In [ ]:
DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = DataFrame['Total_Amount']*0.0096
DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = DataFrame['Total_Amount']*1.17
DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = DataFrame['Total_Amount']*0.70
DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = DataFrame['Total_Amount']*1.29
DataFrame.loc[DataFrame['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [ ]:
DataFrame['Total_Amount'].value_counts()

5.952000e+01    513
4.224000e+01    460
1.728000e+02    351
1.344000e+03    272
1.536000e+01    204
               ... 
1.075200e+04      1
6.869055e+05      1
3.163680e+04      1
1.373772e+06      1
1.749888e+06      1
Name: Total_Amount, Length: 9350, dtype: int64

In [ ]:
DataFrame['Total_Amount'].mean()

1007398.2109417549

##Filtro 


In [ ]:
DataFrame_short = DataFrame[['Region', 'Total_Amount', 'Delivery_Year', 'Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Stage' ]].rename(columns={'Stage': 'Decision'})
DataFrame_short = DataFrame_short[(DataFrame_short['Decision'] == 'Closed Lost') | (DataFrame_short['Decision'] == 'Closed Won')]
DataFrame_short.head()

,Region,Total_Amount,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,EMEA,6169176.0,2016,Q2,1,1,Closed Lost
1,EMEA,56429.1,2016,Q1,0,0,Closed Won
2,Americas,83865.6,2016,Q1,0,0,Closed Won
3,Americas,7421881.5,2018,Q1,0,1,Closed Lost
4,Americas,13357192.5,2018,Q1,0,1,Closed Lost


##Drops de NAN

In [ ]:
DataFrame_short.isnull().sum()

Region                                 0
Total_Amount                          59
Delivery_Year                          0
Delivery_Quarter                       0
Pricing, Delivery_Terms_Approved       0
Pricing, Delivery_Terms_Quote_Appr     0
Decision                               0
dtype: int64

In [ ]:
DataFrame_short.dropna(inplace=True)

In [ ]:
DataFrame_short.isnull().sum()

Region                                0
Total_Amount                          0
Delivery_Year                         0
Delivery_Quarter                      0
Pricing, Delivery_Terms_Approved      0
Pricing, Delivery_Terms_Quote_Appr    0
Decision                              0
dtype: int64

##Filtro total amount mayor a cero y aplico log

In [ ]:
DataFrame_short = DataFrame_short[DataFrame_short["Total_Amount"] > 0 ]
DataFrame_short['Total_Amount'] = np.log(DataFrame_short.Total_Amount)

##Encoding

### **Decision**

In [ ]:
DataFrame_short['Decision'].value_counts()

Closed Won     9527
Closed Lost    7245
Name: Decision, dtype: int64

In [ ]:
DataFrame_short['Decision'] = np.where(DataFrame_short['Decision'] == 'Closed Won',1,0)

In [ ]:
DataFrame_short['Decision'].value_counts()

1    9527
0    7245
Name: Decision, dtype: int64

### **Quarter**

In [ ]:
DataFrame_short['Delivery_Quarter'].value_counts()

Q2    4385
Q1    4264
Q4    4119
Q3    4004
Name: Delivery_Quarter, dtype: int64

In [ ]:
DataFrame_short['Delivery_Quarter_1'] = np.where(DataFrame_short['Delivery_Quarter'] == 'Q1',1,0)

In [ ]:
DataFrame_short['Delivery_Quarter_2'] = np.where(DataFrame_short['Delivery_Quarter'] == 'Q2',1,0)

In [ ]:
DataFrame_short['Delivery_Quarter_3'] = np.where(DataFrame_short['Delivery_Quarter'] == 'Q3',1,0)

In [ ]:
DataFrame_short['Delivery_Quarter_4'] = np.where(DataFrame_short['Delivery_Quarter'] == 'Q4',1,0)

In [ ]:
DataFrame_short.drop('Delivery_Quarter', axis = 1, inplace = True)

### **Region**

In [ ]:
DataFrame_short['Region'].value_counts()

Japan          4845
EMEA           4637
Americas       3892
APAC           3231
Middle East     167
Name: Region, dtype: int64

In [ ]:
DataFrame_short['Region_Japon'] = np.where(DataFrame_short['Region'] == 'Japan',1,0)

In [ ]:
DataFrame_short['Region_EMEA '] = np.where(DataFrame_short['Region'] == 'EMEA',1,0)

In [ ]:
DataFrame_short['Region_Americas'] = np.where(DataFrame_short['Region'] == 'Americas',1,0)

In [ ]:
DataFrame_short['Region_Apac'] = np.where(DataFrame_short['Region'] == 'APAC',1,0)

In [ ]:
DataFrame_short['Region_Middle_East'] = np.where(DataFrame_short['Region'] == 'Middle East',1,0)

In [ ]:
DataFrame_short.drop('Region', axis = 1, inplace = True)

**Df encodeado**

In [ ]:
DataFrame_short.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


##Test

In [ ]:
DataFrame_test1 = pd.read_csv( "/content/Test.csv" )
DataFrame_test1

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,Source_3,South Africa,Account_Name_1686,Opportunity_Name_11120,12364,6508,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_7,4/26/2019,None,None,None,None,None,None,NaT,5/24/2019,4/26/2019,Person_Name_13,Product_Family_213,Product_Name_447,USD,0.300,USD,0.30000,5/1/2019,5/10/2019,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,None,United States,Account_Name_768,Opportunity_Name_9694,12365,6510,Person_Name_38,Person_Name_38,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_4,4/26/2019,None,None,None,None,None,None,N

### **Conversion**

In [ ]:
DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = DataFrame_test1['Total_Amount']*0.0096
DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = DataFrame_test1['Total_Amount']*1.17
DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = DataFrame_test1['Total_Amount']*0.70
DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = DataFrame_test1['Total_Amount']*1.29
DataFrame_test1.loc[DataFrame_test1['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [ ]:
DataFrame_test_short = DataFrame_test1[['Opportunity_ID', 'Region', 'Total_Amount', 'Delivery_Year', 'Delivery_Quarter','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr']].rename(columns={'Stage': 'Decision'})
DataFrame_test_short.head()

,Opportunity_ID,Region,Total_Amount,Delivery_Year,Delivery_Quarter,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
0,10689,EMEA,189820.80,2019,Q2,1,1
1,10689,EMEA,91990.08,2019,Q2,1,1
2,10689,EMEA,148069.35,2019,Q2,1,1
3,10690,EMEA,284731.20,2019,Q2,1,1
4,10690,EMEA,183980.16,2019,Q2,1,1


**log**

In [ ]:
DataFrame_test_short['Total_Amount'] = np.log(DataFrame_test_short.Total_Amount)

### **Quarters**

In [ ]:
DataFrame_test_short['Delivery_Quarter_1'] = np.where(DataFrame_test_short['Delivery_Quarter'] == 'Q1',1,0)

In [ ]:
DataFrame_test_short['Delivery_Quarter_2'] = np.where(DataFrame_test_short['Delivery_Quarter'] == 'Q2',1,0)

In [ ]:
DataFrame_test_short['Delivery_Quarter_3'] = np.where(DataFrame_test_short['Delivery_Quarter'] == 'Q3',1,0)

In [ ]:
DataFrame_test_short['Delivery_Quarter_4'] = np.where(DataFrame_test_short['Delivery_Quarter'] == 'Q4',1,0)

In [ ]:
DataFrame_test_short.drop('Delivery_Quarter', axis = 1, inplace = True)

### **Region**

In [ ]:
DataFrame_test_short['Region_Japon'] = np.where(DataFrame_test_short['Region'] == 'Japan',1,0)

In [ ]:
DataFrame_test_short['Region_EMEA '] = np.where(DataFrame_test_short['Region'] == 'EMEA',1,0)

In [ ]:
DataFrame_test_short['Region_Americas'] = np.where(DataFrame_test_short['Region'] == 'Americas',1,0)

In [ ]:
DataFrame_test_short['Region_Apac'] = np.where(DataFrame_test_short['Region'] == 'APAC',1,0)

In [ ]:
DataFrame_test_short['Region_Middle_East'] = np.where(DataFrame_test_short['Region'] == 'Middle East',1,0)

In [ ]:
DataFrame_test_short.drop('Region', axis = 1, inplace = True)

In [ ]:
DataFrame_test_short.Opportunity_ID = DataFrame_test_short.Opportunity_ID.astype(int)

In [ ]:
DataFrame_test_short = DataFrame_test_short.drop_duplicates('Opportunity_ID',keep = 'last')

In [ ]:
DataFrame_test_short.head()

,Opportunity_ID,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
2,10689,11.905436,2019,1,1,0,1,0,0,0,1,0,0,0
7,10690,11.787392,2019,1,1,0,1,0,0,0,1,0,0,0
8,10691,9.954062,2019,1,1,0,0,0,1,0,0,1,0,0
14,10692,12.338534,2019,1,1,0,0,0,1,0,0,1,0,0
15,10693,8.657390,2019,0,0,0,1,0,0,0,0,1,0,0


In [ ]:
DataFrame_short.head()

,Total_Amount,Delivery_Year,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision,Delivery_Quarter_1,Delivery_Quarter_2,Delivery_Quarter_3,Delivery_Quarter_4,Region_Japon,Region_EMEA,Region_Americas,Region_Apac,Region_Middle_East
0,15.635076,2016,1,1,0,0,1,0,0,0,1,0,0,0
1,10.940740,2016,0,0,1,1,0,0,0,0,1,0,0,0
2,11.336971,2016,0,0,1,1,0,0,0,0,0,1,0,0
3,15.819943,2018,0,1,0,1,0,0,0,0,0,1,0,0
4,16.407566,2018,0,1,0,1,0,0,0,0,0,1,0,0


##Descarga

In [ ]:
DataFrame_short.to_csv('DataFrame_train_encoded_1.csv')
DataFrame_test_short.to_csv('DataFrame_test_encoded_1.csv')

In [ ]:
#files.download('DataFrame_train_encoded_1.csv')
#files.download('DataFrame_test_encoded_1.csv')

#Forma 2 (Column Transformer)

In [ ]:
short_df = DataFrame[['Region','Total_Amount','TRF','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr','Stage' ]].rename(columns={'Stage': 'Decision'})
short_df = short_df[ (short_df['Decision'] == 'Closed Won') | (short_df['Decision'] == 'Closed Lost') ]
short_df['Decision'] = np.where(short_df['Decision'] == 'Closed Won',1,0)
short_df

,Region,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,EMEA,6169176.00,10,1,1,0
1,EMEA,56429.10,0,0,0,1
2,Americas,83865.60,0,0,0,1
3,Americas,7421881.50,14,0,1,0
4,Americas,13357192.50,25,0,1,0
...,...,...,...,...,...,...
16942,EMEA,120919.50,0,1,1,1
16943,EMEA,108827.55,0,1,1,1
16944,EMEA,120919.50,0,1,1,1
16945,Americas,2346796.88,4,1,1,0


In [ ]:
short_df = short_df[short_df['Total_Amount'] > 0]
short_df.describe()

,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
count,1.677200e+04,16772.000000,16772.000000,16772.000000,16772.000000
mean,1.010612e+06,2.281660,0.585440,0.807834,0.568030
std,5.873179e+06,12.084678,0.492661,0.394014,0.495365
min,1.000000e-02,0.000000,0.000000,0.000000,0.000000
25%,4.750453e+03,0.000000,0.000000,1.000000,0.000000
50%,7.714000e+04,0.000000,1.000000,1.000000,1.000000
75%,2.914669e+05,1.000000,1.000000,1.000000,1.000000
max,3.150001e+08,500.000000,1.000000,1.000000,1.000000


In [ ]:
short_df.Total_Amount = np.log(short_df.Total_Amount)
short_df.shape

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


(16772, 6)

In [ ]:
# One-hot-encoding de las variables categóricas
# ==============================================================================
# Se identifica el nobre de las columnas numéricas y categóricas
cat_cols = short_df.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_cols = short_df.select_dtypes(include=['float64', 'int']).columns.to_list()

# Se aplica one-hot-encoding solo a las columnas categóricas
preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough'
               )

# Una vez que se ha definido el objeto ColumnTransformer, con el método fit()
# se aprenden las transformaciones con los datos de entrenamiento y se aplican a
# los dos conjuntos con transform(). Ambas operaciones a la vez con fit_transform().
short_df_prep = preprocessor.fit_transform(short_df)

#El resultado devuelto por ColumnTransformer es un numpy array, por lo que se pierden los nombres de las columnas. Es interesante poder inspeccionar cómo queda el set de datos tras el preprocesado en formato dataframe. Por defecto, OneHotEncoder ordena las nuevas columnas de izquierda a derecha por orden alfabético.

# Convertir el output del ColumnTransformer en dataframe y añadir nombre columnas
# ==============================================================================
# Nombre de todas las columnas
encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names(cat_cols)
labels = np.concatenate([encoded_cat,numeric_cols])

# Conversión a dataframe
short_df_prep = pd.DataFrame(short_df_prep, columns=labels)
short_df_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16772 entries, 0 to 16771
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region_APAC                         16772 non-null  float64
 1   Region_Americas                     16772 non-null  float64
 2   Region_EMEA                         16772 non-null  float64
 3   Region_Japan                        16772 non-null  float64
 4   Region_Middle East                  16772 non-null  float64
 5   Total_Amount                        16772 non-null  float64
 6   TRF                                 16772 non-null  float64
 7   Pricing, Delivery_Terms_Approved    16772 non-null  float64
 8   Pricing, Delivery_Terms_Quote_Appr  16772 non-null  float64
 9   Decision                            16772 non-null  float64
dtypes: float64(10)
memory usage: 1.3 MB


In [ ]:
short_df_prep.head()

,Region_APAC,Region_Americas,Region_EMEA,Region_Japan,Region_Middle East,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr",Decision
0,0.0,0.0,1.0,0.0,0.0,15.635076,10.0,1.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,10.940740,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,11.336971,0.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,15.819943,14.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0,16.407566,25.0,0.0,1.0,0.0


In [ ]:
short_df_prep.to_csv('short_df_Encoded.csv')

## Test

In [ ]:
DataFrame_test = pd.read_csv( "/content/Test.csv" )
DataFrame_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,Source_3,South Africa,Account_Name_1686,Opportunity_Name_11120,12364,6508,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_7,4/26/2019,None,None,None,None,None,None,NaT,5/24/2019,4/26/2019,Person_Name_13,Product_Family_213,Product_Name_447,USD,0.300,USD,0.30000,5/1/2019,5/10/2019,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,None,United States,Account_Name_768,Opportunity_Name_9694,12365,6510,Person_Name_38,Person_Name_38,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_4,4/26/2019,None,None,None,None,None,None,N

In [ ]:
DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = DataFrame_test['Total_Amount']*0.0096
DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = DataFrame_test['Total_Amount']*1.17
DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = DataFrame_test['Total_Amount']*0.70
DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = DataFrame_test['Total_Amount']*1.29
DataFrame_test.loc[DataFrame_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

In [ ]:
DataFrame_test = DataFrame_test[['Opportunity_ID','Region','Total_Amount','TRF','Pricing, Delivery_Terms_Approved','Pricing, Delivery_Terms_Quote_Appr' ]]
DataFrame_test = DataFrame_test.drop_duplicates('Opportunity_ID',keep = 'last')
DataFrame_test

,Opportunity_ID,Region,Total_Amount,TRF,"Pricing, Delivery_Terms_Approved","Pricing, Delivery_Terms_Quote_Appr"
2,10689,EMEA,148069.350,0,1,1
7,10690,EMEA,131582.880,0,1,1
8,10691,Americas,21037.500,0,1,1
14,10692,Americas,228327.000,1,1,1
15,10693,Americas,5752.500,0,0,0
...,...,...,...,...,...,...
2546,12364,EMEA,124740.000,0,1,1
2547,12365,Americas,45054.900,0,1,1
2548,12366,Americas,100122.000,0,1,1
2549,12367,Americas,143220.000,0,1,1


In [ ]:
DataFrame_test.Total_Amount = np.log(DataFrame_test.Total_Amount)
DataFrame_test['Total_Amount'].describe()

count    1567.000000
mean       11.581272
std         1.843534
min         4.516018
25%        10.700326
50%        11.664994
75%        12.496506
max        19.996115
Name: Total_Amount, dtype: float64

In [ ]:
# One-hot-encoding de las variables categóricas
# ==============================================================================
# Se identifica el nobre de las columnas numéricas y categóricas
cat_cols = DataFrame_test.select_dtypes(include=['object', 'category']).columns.to_list()
numeric_cols = DataFrame_test.select_dtypes(include=['float64', 'int']).columns.to_list()

# Se aplica one-hot-encoding solo a las columnas categóricas
preprocessor = ColumnTransformer(
                    [('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
                    remainder='passthrough'
               )

# Una vez que se ha definido el objeto ColumnTransformer, con el método fit()
# se aprenden las transformaciones con los datos de entrenamiento y se aplican a
# los dos conjuntos con transform(). Ambas operaciones a la vez con fit_transform().
DataFrame_test_prep = preprocessor.fit_transform(DataFrame_test)

#El resultado devuelto por ColumnTransformer es un numpy array, por lo que se pierden los nombres de las columnas. Es interesante poder inspeccionar cómo queda el set de datos tras el preprocesado en formato dataframe. Por defecto, OneHotEncoder ordena las nuevas columnas de izquierda a derecha por orden alfabético.

# Convertir el output del ColumnTransformer en dataframe y añadir nombre columnas
# ==============================================================================
# Nombre de todas las columnas
encoded_cat = preprocessor.named_transformers_['onehot'].get_feature_names(cat_cols)
labels = np.concatenate([encoded_cat, numeric_cols])

# Conversión a dataframe
DataFrame_test_prep = pd.DataFrame(DataFrame_test_prep, columns=labels)
DataFrame_test_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 10 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Region_APAC                         1567 non-null   float64
 1   Region_Americas                     1567 non-null   float64
 2   Region_EMEA                         1567 non-null   float64
 3   Region_Japan                        1567 non-null   float64
 4   Region_Middle East                  1567 non-null   float64
 5   Opportunity_ID                      1567 non-null   float64
 6   Total_Amount                        1567 non-null   float64
 7   TRF                                 1567 non-null   float64
 8   Pricing, Delivery_Terms_Approved    1567 non-null   float64
 9   Pricing, Delivery_Terms_Quote_Appr  1567 non-null   float64
dtypes: float64(10)
memory usage: 122.5 KB


In [ ]:
DataFrame_test_prep.Opportunity_ID = DataFrame_test_prep.Opportunity_ID.astype(int)

In [ ]:
DataFrame_test_prep = DataFrame_test_prep.drop_duplicates('Opportunity_ID',keep = 'last')

In [ ]:
DataFrame_test_prep.to_csv('DataFrame_test_encoded.csv')

In [ ]:
#files.download('short_df_Encoded.csv')
#files.download('DataFrame_test_encoded.csv')

#Forma 3 (Mean Encoding) con otras columnas

In [ ]:
df3 = pd.read_csv( "/content/Training.csv" )

In [ ]:
df4.loc[df4['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df4['Total_Amount']*0.0096
df4.loc[df4['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df4.loc[df4['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df4['Total_Amount']*1.17
df4.loc[df4['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] = 'USD'

df4.loc[df4['Total_Amount_Currency'] == 'AUD', 'Total_Amount'] = df4['Total_Amount']*0.70
df4.loc[df4['Total_Amount_Currency'] == 'AUD', 'Total_Amount_Currency'] = 'USD'

df4.loc[df4['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df4['Total_Amount']*1.29
df4.loc[df4['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'
df4.dropna(inplace = True)
df4['Opportunity_Created_Date'] = pd.to_datetime(df4['Opportunity_Created_Date'],errors='coerce')
df4["Year Created"] = df4["Opportunity_Created_Date"].dt.year
df4["Month Created"] =df4["Opportunity_Created_Date"].dt.month
df4["Expensive"] = np.where(df4["Total_Taxable_Amount"] >= 80000, 1, 0)
df4['Planned_Delivery_Start_Date'] = pd.to_datetime(df4['Planned_Delivery_Start_Date'],errors='coerce')
df4['Planned_Delivery_End_Date'] = pd.to_datetime(df4['Planned_Delivery_End_Date'],errors='coerce')
df4 = df4[df4["Total_Amount"] > 0]
df4["Total_Amount"] = np.log(df4["Total_Amount"])
entrenamiento_by_region = df4.groupby("Region").agg({"Stage" : "count"})
entrenamiento_by_region.rename(columns = {"Stage" : "Amount of Cases"}, inplace = True)
entrenamiento_won_by_region = df4[df4["Stage"] == "Closed Won"].groupby("Region").agg({"Stage" : "count"})
entrenamiento_won_by_region.rename(columns = {"Stage" : "Cases won"}, inplace = True)

entrenamiento_won_by_region["percentage"] = (entrenamiento_won_by_region["Cases won"] / entrenamiento_by_region["Amount of Cases"]) * 100
df4 = df4.merge(entrenamiento_won_by_region, on = "Region")
df4["Años en entregar"] = df4["Delivery_Year"] - df4["Year Created"]

In [ ]:
short_df4 = df4[['Opportunity_ID', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner", 'Stage' ]].rename(columns={'Stage': 'Decision'})
short_df4 = short_df4[ (short_df4['Decision'] == 'Closed Won') | (short_df4['Decision'] == 'Closed Lost') ]
short_df4['Decision'] = np.where(short_df4['Decision'] == 'Closed Won',1,0)
short_df4.set_index('Opportunity_ID', inplace = True)


short_df4.describe()
short_df4.dropna(inplace = True)

In [ ]:
short_df4["Product_Name"] = np.where(short_df4.groupby('Product_Name')["Product_Name"].transform(len) > 5, short_df4["Product_Name"], "Other")
short_df4["Product_Name"].value_counts()

Product_Name_100    968
Product_Name_226    835
Product_Name_231    574
Product_Name_192    437
Other               397
                   ... 
Product_Name_264      6
Product_Name_254      6
Product_Name_106      6
Product_Name_17       6
Product_Name_388      6
Name: Product_Name, Length: 237, dtype: int64

In [ ]:
short_df4["Opportunity_Owner"] = np.where(short_df4.groupby('Opportunity_Owner')["Opportunity_Owner"].transform(len) > 5, short_df4["Opportunity_Owner"], "Other")
short_df4["Opportunity_Owner"].value_counts()

Person_Name_8     1352
Person_Name_13    1252
Person_Name_50    1071
Person_Name_18     979
Person_Name_43     831
Person_Name_3      760
Person_Name_4      682
Person_Name_32     655
Person_Name_19     614
Person_Name_16     438
Person_Name_23     422
Person_Name_64     408
Person_Name_65     322
Person_Name_49     285
Person_Name_46     238
Person_Name_44     225
Person_Name_66     218
Person_Name_39     210
Person_Name_60     206
Person_Name_48     198
Person_Name_38     193
Person_Name_62     174
Person_Name_52     173
Person_Name_54     168
Person_Name_5      152
Person_Name_59     134
Person_Name_2      118
Person_Name_45     112
Person_Name_41     112
Person_Name_51     111
Person_Name_33      86
Person_Name_14      82
Person_Name_55      79
Person_Name_11      70
Person_Name_30      64
Person_Name_63      61
Person_Name_40      50
Person_Name_25      48
Person_Name_20      48
Person_Name_26      33
Person_Name_22      33
Person_Name_37      21
Person_Name_12      19
Person_Name

In [ ]:
mean_encoded_owner = short_df4.groupby("Opportunity_Owner")["Decision"].mean().to_dict()
short_df4["Opportunity_Owner"] = short_df4["Opportunity_Owner"].map(mean_encoded_owner)

In [ ]:
mean_encoded_product = short_df4.groupby("Product_Name")["Decision"].mean().to_dict()

In [ ]:
short_df4["Product_Name"] = short_df4["Product_Name"].map(mean_encoded_product)

In [ ]:
short_df4.head()

,Total_Amount,Años en entregar,Month Created,Product_Name,Opportunity_Owner,Decision
Opportunity_ID,,,,,,
0,15.635076,1,12,0.186992,0.560776,0
1,10.940740,1,12,0.592975,0.750000,1
19,12.624960,1,12,0.561404,0.261905,0
20,11.702880,1,12,0.592975,0.664537,1
23,12.433069,1,12,0.592975,0.560776,1


##Test

In [ ]:
df4_test = pd.read_csv( "/content/Test.csv" )
df4_test

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,162240.0,EUR,367419.0,Prod_Category_A_None
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,78624.0,EUR,367419.0,Prod_Category_A_None
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,126555.0,EUR,367419.0,Prod_Category_A_None
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,243360.0,EUR,757783.5,Prod_Category_A_None
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,4/12/2019,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,157248.0,EUR,757783.5,Prod_Category_A_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,16345,EMEA,KSA,1,1,0,0,0,Bureaucratic_Code_4,6/12/2017,Source_3,South Africa,Account_Name_1686,Opportunity_Name_11120,12364,6508,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_7,4/26/2019,None,None,None,None,None,None,NaT,5/24/2019,4/26/2019,Person_Name_13,Product_Family_213,Product_Name_447,USD,0.300,USD,0.30000,5/1/2019,5/10/2019,2019 - 5,Q2,2019,NaT,0,USD,124740.0,USD,147750.0,Prod_Category_A_None
2547,15218,Americas,SE America,1,1,0,0,0,Bureaucratic_Code_4,6/8/2018,None,United States,Account_Name_768,Opportunity_Name_9694,12365,6510,Person_Name_38,Person_Name_38,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_4,4/26/2019,None,None,None,None,None,None,N

In [ ]:
df4_test.loc[df4_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount'] = df4_test['Total_Amount']*0.0096
df4_test.loc[df4_test['Total_Amount_Currency'] == 'JPY', 'Total_Amount_Currency'] = 'USD'

df4_test.loc[df4_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount'] = df4_test['Total_Amount']*1.17
df4_test.loc[df4_test['Total_Amount_Currency'] == 'EUR', 'Total_Amount_Currency'] 
df4_test.loc[df4_test['Total_Amount_Currency'] == 'AUD', 'Total_Taxable_Amount'] = df4_test['Total_Amount']*0.70
df4_test.loc[df4_test['Total_Amount_Currency'] == 'AUD', 'Total_Taxable_Amount_Currency'] = 'USD'

df4_test.loc[df4_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount'] = df4_test['Total_Amount']*1.29
df4_test.loc[df4_test['Total_Amount_Currency'] == 'GBP', 'Total_Amount_Currency'] = 'USD'

df4_test['Opportunity_Created_Date'] = pd.to_datetime(df4_test['Opportunity_Created_Date'],errors='coerce')

df4_test["Year Created"] = df4_test["Opportunity_Created_Date"].dt.year
df4_test["Month Created"] =df4_test["Opportunity_Created_Date"].dt.month
df4_test["Expensive"] = np.where(df4_test["Total_Amount"] >= 80000, 1, 0)


df4_test = df4_test.merge(entrenamiento_won_by_region, on = "Region")
df4_test["Años en entregar"] = df4_test["Delivery_Year"] - df4_test["Year Created"]
df4_test["Total_Taxable_Amount"].describe()

count    2.551000e+03
mean     3.393353e+06
std      2.430590e+07
min      0.000000e+00
25%      6.987750e+04
50%      2.195684e+05
75%      7.818624e+05
max      6.808725e+08
Name: Total_Taxable_Amount, dtype: float64

In [ ]:
df4_test.head()

,ID,Region,Territory,"Pricing, Delivery_Terms_Quote_Appr","Pricing, Delivery_Terms_Approved",Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Submitted_for_Approval,Bureaucratic_Code,Account_Created_Date,Source,Billing_Country,Account_Name,Opportunity_Name,Opportunity_ID,Sales_Contract_No,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Opportunity_Created_Date,Brand,Product_Type,Size,Product_Category_B,Price,Currency,Last_Activity,Quote_Expiry_Date,Last_Modified_Date,Last_Modified_By,Product_Family,Product_Name,ASP_Currency,ASP,ASP_(converted)_Currency,ASP_(converted),Planned_Delivery_Start_Date,Planned_Delivery_End_Date,Month,Delivery_Quarter,Delivery_Year,Actual_Delivery_Date,TRF,Total_Amount_Currency,Total_Amount,Total_Taxable_Amount_Currency,Total_Taxable_Amount,Prod_Category_A,Year Created,Month Created,Expensive,Cases won,percentage,Años en entregar
0,6140,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_171,Product_Name_332,EUR,0.325,USD,0.36761,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,189820.80,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
1,6146,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,91990.08,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
2,6151,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6118,10689,6436,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/12/2019,Person_Name_13,Product_Family_140,Product_Name_241,EUR,0.275,USD,0.31105,5/1/2019,5/31/2019,2019 - 5,Q2,2019,NaT,0,EUR,148069.35,EUR,367419.0,Prod_Category_A_None,2019,4,1,2467,53.179565,0
3,6118,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_171,Product_Name_328,EUR,0.325,USD,0.36761,6/1/2019,6/28/2019,2019 - 6,Q2,2019,NaT,1,EUR,284731.20,EUR,757783.5,Prod_Category_A_None,2019,4,1,2467,53.179565,0
4,6124,EMEA,Germany,1,1,1,1,0,Bureaucratic_Code_4,7/5/2017,Source_7,Germany,Account_Name_978,Opportunity_Name_6117,10690,6504,Person_Name_13,Person_Name_13,Account_Type_0,Opportunity_Type_8,Non Binding,Delivery_Terms_2,2019-04-12,None,None,None,None,None,None,NaT,4/30/2019,4/25/2019,Person_Name_13,Product_Family_167,Product_Name_315,EUR,0.320,USD,0.36195,6/1/2019,6/29/2019,2019 - 6,Q2,2019,NaT,0,EUR,183980.16,EUR,757783.5,Prod_Category_A_None,2019,4,1,2467,53.179565,0


In [ ]:
df4_test["Total_Amount"] = np.log(df4_test["Total_Amount"])
df4_test = df4_test[['Opportunity_ID', "Total_Amount", "Años en entregar","Month Created", "Product_Name", "Opportunity_Owner"]]
df4_test = df4_test.drop_duplicates('Opportunity_ID',keep = 'first')
df4_test.set_index('Opportunity_ID', inplace = True)
df4_test["Product_Name"] = df4_test["Product_Name"].map(mean_encoded_product)
df4_test["Opportunity_Owner"] = df4_test["Opportunity_Owner"].map(mean_encoded_owner)

In [ ]:
df4_test["Product_Name"].fillna(mean_encoded_product["Other"], inplace = True)
df4_test["Opportunity_Owner"].fillna(mean_encoded_owner["Other"], inplace = True)

In [ ]:
short_df4.to_csv('short_df_train_encoded_3.csv')
df4_test.to_csv('short_df_test_encoded_3.csv')

In [ ]:
files.download('short_df_train_encoded_3.csv')
files.download('short_df_test_encoded_3.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>